In [1]:
#Install all the Reqiuired Libraries and Packages
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
# from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
# import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
#All the Required Packages and Libraies are installed.
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=37901e82621e1f9f3c847112ac0d224af4fd1459d842aa7c5e3fb5a9976383f0
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [3]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import librosa as lr
import os, glob, pickle

# Verify the installation by importing the package
print("python_speech_features installed and imported successfully!")


python_speech_features installed and imported successfully!


In [4]:
# Step 1: Install the package
!pip install python_speech_features

# Step 2: Import the package and other dependencies
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import librosa as lr
import os, glob, pickle

# Verify the installation by importing the package
print("python_speech_features installed and imported successfully!")

# Additional code for your project can be added here
# For example, loading and processing audio files, etc.


python_speech_features installed and imported successfully!


In [5]:
import os

def getListOfFiles(dirName):
    # Create a list of files and subdirectories
    listOfFile = os.listdir(dirName)
    allFiles = list()

    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory, recursively get the list of files
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

# Define the path to the directory in Google Drive
dirName = '/content/drive/MyDrive/Audio_Speech_Actors_01-24'

# Get the list of all files in the directory and its subdirectories
listOfFiles = getListOfFiles(dirName)

# Print the number of files
print("Number of files:", len(listOfFiles))


Number of files: 1482


In [6]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 34.3 MB/s eta 0:00:00


In [7]:
import speech_recognition as sr

# Initialize the recognizer
r = sr.Recognizer()

# Process each audio file and recognize text
for file in range(len(listOfFiles)):
    with sr.AudioFile(listOfFiles[file]) as source:
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            print(f"File: {listOfFiles[file]}\nText: {text}\n")
        except Exception as e:
            print(f"File: {listOfFiles[file]}\nError: {e}\n")


File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-01-01-01-02-24 (1).wav
Text: kids are talking by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-01-01-02-01-24.wav
Text: dogs are sitting by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-02-01-01-02-24.wav
Text: kids are talking by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-01-01-02-02-24.wav
Text: dogs are sitting by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-02-01-02-01-24 (1).wav
Text: dogs are sitting by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-02-01-01-01-24.wav
Text: kids are talking by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-01-01-01-01-24 (1).wav
Text: kids are talking by the door

File: /content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_24/03-01-01-01-01-02-24.wav
Text: kids are talking b

In [9]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask


In [10]:
import soundfile as sf

In [11]:
# Function to list all files in a directory recursively
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

# Define the path to the directory in Google Drive
dirName = '/content/drive/MyDrive/Audio_Speech_Actors_01-24'

# Get the list of all files in the directory and its subdirectories
listOfFiles = getListOfFiles(dirName)

# Create directory for cleaned audio files
cleaned_dir = '/content/drive/My Drive/speech-emotion-recognition-cleaned-data'
if not os.path.exists(cleaned_dir):
    os.makedirs(cleaned_dir)

# Process each audio file
for file in listOfFiles:
    # Load the audio file
    y, sr = librosa.load(file, sr=None)

    # Apply the envelope function to create a mask
    mask = envelope(y, sr, 0.0005)

    # Downsample the audio
    y_clean = y[mask]

    # Define the path for the cleaned audio file
    file_name = os.path.basename(file)
    cleaned_file_path = os.path.join(cleaned_dir, file_name)

    # Save the cleaned audio file
    sf.write(cleaned_file_path, y_clean, sr)

print("Audio files have been cleaned and saved.")


Audio files have been cleaned and saved.


In [13]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result



Define extract_feature Function

In [20]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

        if len(X) < 512:
            return None  # Skip files that are too short

        result = np.array([])

        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))

        if chroma:
            stft = np.abs(librosa.stft(X, n_fft=512))
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))

        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate, n_fft=512).T, axis=0)
            result = np.hstack((result, mel))

    return result


Get List of Files and Extract Features

In [21]:
# # Define the path to the directory in Google Drive
# dirName = '/content/drive/MyDrive/Audio_Speech_Actors_01-24'

# # Get the list of all files in the directory and its subdirectories
# def getListOfFiles(dirName):
#     listOfFile = os.listdir(dirName)
#     allFiles = list()
#     for entry in listOfFile:
#         fullPath = os.path.join(dirName, entry)
#         if os.path.isdir(fullPath):
#             allFiles = allFiles + getListOfFiles(fullPath)
#         else:
#             allFiles.append(fullPath)
#     return allFiles

# listOfFiles = getListOfFiles(dirName)

# # Extract features from each audio file
# features = []
# for file in listOfFiles:
#     feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
#     if feature is not None:
#         features.append(feature)

# print("Feature extraction completed.")


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [22]:
# Define the path to the directory in Google Drive
dirName = '/content/drive/MyDrive/Audio_Speech_Actors_01-24'

# Get the list of all files in the directory and its subdirectories
def getListOfFiles(dirName):
    listOfFiles = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFiles:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles.extend(getListOfFiles(fullPath))  # Use extend for better performance
        else:
            allFiles.append(fullPath)
    return allFiles

listOfFiles = getListOfFiles(dirName)

# Improved extract_feature function (assuming you're using librosa)
import librosa  # Assuming librosa for feature extraction

def extract_feature(file, mfcc=True, chroma=True, mel=True):
    y, sr = librosa.load(file)  # Load audio data
    features = []

    # Extract features conditionally to ensure consistent shape
    if mfcc:
        mfccs = librosa.feature.mfcc(y=y, sr=sr)
        features.append(mfccs)
    if chroma:
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        features.append(chroma_stft)
    if mel:
        melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        features.append(melspectrogram)

    # Concatenate features only if all have the same shape
    if len(features) > 1 and all(feat.shape == features[0].shape for feat in features[1:]):
        return np.concatenate(features, axis=0)  # Concatenate along axis 0 (rows)
    else:
        print(f"Warning: Features from '{file}' have different shapes. Skipping.")
        return None  # Indicate that features couldn't be combined

features = []
for file in listOfFiles:
    feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
    if feature is not None:
        features.append(feature)

print("Feature extraction completed.")


Feature extraction completed.


Define the Emotions and Observed Emotions

In [29]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

        if len(X) < 512:
            return None  # Skip files that are too short

        result = np.array([])

        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))

        if chroma:
            stft = np.abs(librosa.stft(X, n_fft=512))
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))

        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate, n_fft=512).T, axis=0)
            result = np.hstack((result, mel))

    return result


In [30]:
#Emotions in the RAVDESS dataset to be classified Audio Files based on .
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#These are the emotions User wants to observe more :
observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [40]:
# from os.path import basename, join
# from glob import glob
# from sklearn.model_selection import train_test_split

# dirName = '/content/drive/MyDrive/Audio_Speech_Actors_01-24'  # Replace with your actual path
# X_train, X_test, y_train, y_test = load_data(dirName, emotions)


# # Assuming you have defined functions `extract_feature` and `emotions` elsewhere

# def load_data(data_dir, emotions, observed_emotions=None, test_size=0.33):
#   """
#   Loads audio data and extracts features for each sound file.

#   Args:
#       data_dir (str): Path to the directory containing audio files.
#       emotions (dict): Dictionary mapping filenames (without extension) to emotions.
#       observed_emotions (list, optional): List of emotions to consider (filters out others). Defaults to None (all emotions).
#       test_size (float, optional): Proportion of data to use for testing. Defaults to 0.33.

#   Returns:
#       tuple: Tuple containing training and testing data (X_train, X_test, y_train, y_test).
#   """

#   x, y = [], []

#   for file in glob(join(dirName, "*.wav")):
#     # Extract filename without extension
#     file_name = basename(file)

#     # Check if emotion exists and is in the observed list (if provided)
#     emotion = emotions.get(file_name.split("-")[2])
#     if emotion is None or (observed_emotions is not None and emotion not in observed_emotions):
#       continue  # Skip files with unknown or unobserved emotions

#     # Extract features
#     feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
#     x.append(feature)
#     y.append(emotion)  # Append only the emotion label

#   # Convert lists to NumPy arrays
#   X = np.array(x)
#   y = np.array(y)

#   # Split data into training and testing sets
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

#   return X_train, X_test, y_train, y_test

# # Example usage (assuming you have defined data_dir and emotions dictionary)
# X_train, X_test, y_train, y_test = load_data(data_dir, emotions)
# print("Training data shapes:", X_train.shape, y_train.shape)
# print("Testing data shapes:", X_test.shape, y_test.shape)


In [42]:
def load_data(dirName, emotions, observed_emotions=None, test_size=0.33):
  """
  Loads audio data and extracts features for each sound file.

  Args:
      data_dir (str): Path to the directory containing audio files (WAV format assumed).
      emotions (dict): Dictionary mapping filenames (without extension) to emotions.
      observed_emotions (list, optional): List of emotions to consider (filters out others). Defaults to None (all emotions).
      test_size (float, optional): Proportion of data to use for testing. Defaults to 0.33.

  Returns:
      tuple: Tuple containing training and testing data (X_train, X_test, y_train, y_test).
  """

  x, y = [], []

  for file in glob(join(dirName, "*.wav")):
    # Extract filename without extension
    file_name = basename(file)

    # Check if emotion exists and is in the observed list (if provided)
    emotion = emotions.get(file_name.split("-")[2])
    if emotion is None or (observed_emotions is not None and emotion not in observed_emotions):
      continue  # Skip files with unknown or unobserved emotions

    # Try-except block for handling potential feature extraction errors
    try:
      # Extract features (assuming you're using librosa)
      feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
      if feature is not None:  # Check if feature extraction was successful
        x.append(feature)
        y.append(emotion)  # Append only the emotion label
    except Exception as e:
      print(f"Error processing file '{file_name}': {e}")
      continue  # Skip files with extraction errors

  # Convert lists to NumPy arrays
  X = np.array(x)
  y = np.array(y)

  # Split data into training and testing sets (ensure X and y have same length)
  if len(X) > 0:  # Check if any features were extracted before splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
  else:
    print("No valid audio files or emotions found. Please check your data directory and emotion mapping.")
    # Return None for all outputs if no data
    return None, None, None, None  # Corrected: Inside the function

  return X_train, X_test, y_train, y_test

# Assuming you have defined `dirName` and `emotions` dictionary
X_train, X_test, y_train, y_test = load_data(dirName, emotions)


No valid audio files or emotions found. Please check your data directory and emotion mapping.
